In [14]:
import numpy as np
a = np.load('test.npy')
print(a.shape)
print(a.reshape(16,-1,order='F'))

(8, 2, 10, 2)
[[   0.            0.            0.            1.            0.
     0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.        ]
 [   0.            0.            0.            0.            1.
     0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.        ]
 [   0.            0.            0.            0.            0.
     1.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.        ]
 [ -23.80888889   11.82222222    0.           -0.30133333    0.
     0.           11.98666667    0.            0.            0.
     0.            0.            0.            0.            0.